In [63]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import seaborn as sns
import pickle
from tqdm.auto import tqdm
import gensim.corpora as corpora
import pandas as pd
import numpy as np

In [64]:
lda_model=pickle.load(open("output/lda_model/lda_22_5172.pkl","rb"))
all_original_text=pickle.load(open("Data/all_original_text.pkl","rb"))

In [65]:
len(lda_model.id2word)

5561

In [66]:
class lda_vis:

    @staticmethod
    def check_corpus(corpus,dict):
        result=pd.DataFrame(
            [
                list(range(len(dict))),
                [0]*len(dict)
            ]
        ).T
        result.columns=["word","frequency"]
        for post in tqdm(corpus,desc="Checking Corpus"):
            for word_tuple in post:
                result.loc[word_tuple[0],"frequency"]+=word_tuple[1]
        for i in result.index:
            result.loc[i,"word"]=dict[i]
        result=result.sort_values(by="frequency",ascending=False)
        return result

    def __init__(self):
        self.model=lda_model
        self.text=all_original_text
        self.dict=self.model.id2word
        self.corpus=[
            self.dict.doc2bow(post)
            for post in self.text
        ]
        self.topic_word_distribution=''
        for topic_id,word_with_weights in self.model.show_topics(num_topics=-1, num_words=10, formatted=False):
            self.topic_word_distribution+=f"Topic {topic_id+1}:\n"
            for word,weight in word_with_weights:
                self.topic_word_distribution+=f"{word}:{weight:.4f}\n"
            self.topic_word_distribution+="\n"
    
    def get_word_frequency(self):
        self.word_frequency=self.check_corpus(self.corpus,self.dict)
        return self.word_frequency
    

In [67]:
lda_22_5172=lda_vis()

In [68]:
print(lda_22_5172.topic_word_distribution)

Topic 1:
country:0.0052
state:0.0052
american:0.0047
gun:0.0044
law:0.0041
vote:0.0039
america:0.0036
party:0.0034
abortion:0.0031
government:0.0031

Topic 2:
episode:0.0175
season:0.0113
anime:0.0101
series:0.0096
scene:0.0088
unnie:0.0074
manga:0.0060
ep:0.0038
version:0.0031
film:0.0031

Topic 3:
group:0.0072
stalker:0.0071
member:0.0070
kpop:0.0065
comeback:0.0062
yall:0.0060
devote:0.0058
bias:0.0054
mark:0.0048
nct:0.0047

Topic 4:
sell:0.0202
ticket:0.0200
order:0.0154
concert:0.0150
pc:0.0110
tour:0.0105
harry:0.0104
direct:0.0097
item:0.0090
update:0.0080

Topic 5:
experience:0.0041
photo:0.0029
plan:0.0029
appreciate:0.0028
past:0.0026
spend:0.0026
join:0.0026
grow:0.0026
create:0.0025
amaze:0.0025

Topic 6:
stream:0.0278
draw:0.0062
smile:0.0061
adorable:0.0054
join:0.0052
hug:0.0050
kiss:0.0050
twitch:0.0038
direct:0.0037
commission:0.0037

Topic 7:
bts:0.0517
automatically:0.0455
unfollowed:0.0344
army:0.0233
jungkook:0.0218
yoongi:0.0200
jimin:0.0197
hobi:0.0194
taehyung: